<a href="https://colab.research.google.com/github/sotetsuk/pgx/blob/harukaki%2Ffeat%2Fbridge_dds_learning/workspace/dds_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!pip install optax
!pip install tqdm
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 KB 12.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=685d2807

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import sqlite3
from contextlib import closing
import pandas as pd
import json
import numpy as np
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import time
import tqdm
import os
import wandb
import math

In [5]:
wandb.login()
wandb.init(project = "dds_learning",
           config = {
                  "TRAIN_SAMPLE_NUM" : 2000000,
                  "TEST_SAMPLE_NUM" : 500000,
                  "EPOCHS" : 10,
                  "BATCH_SIZE" : 1000,
                  "HIDDEN_LAYER_SIZE" : 1024,
                  "LAYER_NUM" : 5,
                  "lr" : 1e-4}
           )
config = wandb.config

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hrkkt1213. Use `wandb login --relogin` to force relogin


In [6]:

def pbn_to_bool(pbn):
  deal = [0] * 208
  suit = 0
  for i in range(9, 76):
    if pbn[i] == " ":
      suit += 13
    elif pbn[i] == ".":
      suit += 13
    elif pbn[i] == "A":
      deal[suit] = 1
    elif pbn[i] == "K":
      deal[suit + 12] = 1
    elif pbn[i] == "Q":
      deal[suit + 11] = 1
    elif pbn[i] == "J":
      deal[suit + 10] = 1
    elif pbn[i] == "T":
      deal[suit + 9] = 1
    else:
      deal[suit + int(pbn[i]) - 1] = 1
  return deal


if os.path.exists('/content/drive/MyDrive/original/train_deals.npy'):
  print("files exist")
  print("load files")
  train_deals = jnp.array(np.load('/content/drive/MyDrive/original/train_deals.npy')[: config.TRAIN_SAMPLE_NUM, :], dtype=jnp.float32)
  train_ddts = jnp.array(np.load('/content/drive/MyDrive/original/train_ddts.npy')[: config.TRAIN_SAMPLE_NUM, :], dtype=jnp.float32)
  test_deals = jnp.array(np.load('/content/drive/MyDrive/original/test_deals.npy')[: config.TEST_SAMPLE_NUM, :], dtype=jnp.float32)
  test_ddts = jnp.array(np.load('/content/drive/MyDrive/original/test_ddts.npy')[: config.TEST_SAMPLE_NUM, :], dtype=jnp.float32)


else:
  print("files don't exist")
  print("make files")
  dbname = "/content/drive/MyDrive/original/dda.db"
  with sqlite3.connect(dbname) as conn:
    df = pd.read_sql('select * from records', con=conn)
  display(df)
  train_deals = []
  train_ddts = []
  print("START : make train dataset")
  for i in tqdm.tqdm(range(config.TRAIN_SAMPLE_NUM)):
    
    content = json.loads(df.loc[i, "CONTENT"])
    train_deals.append(pbn_to_bool(content["pbn"]))
    train_ddts.append(content["ddt"])

    
  test_deals = []
  test_ddts = []   
  print("START : make test dataset")
  for i in tqdm.tqdm(range(config.TRAIN_SAMPLE_NUM, config.TRAIN_SAMPLE_NUM + config.TEST_SAMPLE_NUM)):
    content = json.loads(df.loc[i, "CONTENT"])
    test_deals.append(pbn_to_bool(content["pbn"]))
    test_ddts.append(content["ddt"])

  train_deals = np.array(train_deals)
  train_ddts = np.array(train_ddts)
  test_deals = np.array(test_deals)
  test_ddts = np.array(test_ddts)
  np.save('/content/drive/MyDrive/original/train_deals.npy', train_deals)
  np.save('/content/drive/MyDrive/original/train_ddts.npy', train_ddts)
  np.save('/content/drive/MyDrive/original/test_deals.npy', test_deals)
  np.save('/content/drive/MyDrive/original/test_ddts.npy', test_ddts)
  train_deals = jnp.array(train_deals, dtype=jnp.float32)
  train_ddts = jnp.array(train_ddts, dtype=jnp.float32)
  test_deals = jnp.array(test_deals, dtype=jnp.float32)
  test_ddts = jnp.array(test_ddts, dtype=jnp.float32)
print(jnp.shape(train_deals))
print(jnp.shape(train_ddts))
print(jnp.shape(test_deals))
print(jnp.shape(test_ddts))




files exist
load files


(2000000, 208)
(2000000, 20)
(500000, 208)
(500000, 20)


In [7]:
# ヘルパー関数
# dense NNのパラメータをランダム値で初期化
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n, ))

# "sizes"をサイズにもつ全結合層の全ての層を初期化
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  print(len(sizes))
  print(sizes[:-1])
  print(sizes[1:])
  print(keys)
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1: ], keys)]


layer_sizes = [208]
for i in range(config.LAYER_NUM):
  layer_sizes.append(config.HIDDEN_LAYER_SIZE)
layer_sizes.append(20)
step_size = 0.01
print(random.PRNGKey(0))
params = init_network_params(layer_sizes, random.PRNGKey(0))
print(jnp.shape(params))

[0 0]
7
[208, 1024, 1024, 1024, 1024, 1024]
[1024, 1024, 1024, 1024, 1024, 20]
[[2716826189 1180506676]
 [4247964747 2620019508]
 [4167348494   78606281]
 [2231548119  292468403]
 [1537975273  917206909]
 [3902452147 3915755166]
 [3865067960 3800283793]]
(6, 2)


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [8]:
from jax.scipy.special import logsumexp
def relu(x):
  return jnp.maximum(0, x)

def predict(params, deal):
  # 一個ごとの予測
  activations = deal
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = relu(outputs)
  
  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b
  return logits 

# Make a batched version of the `predict` function
batched_predict = vmap(predict, in_axes=(None, 0))

In [9]:
import optax


optimizer = optax.adam(config.lr)
opt_state = optimizer.init(params)

In [10]:
def loss(params, deals, ddts):
  preds = batched_predict(params, deals)
  return jnp.mean((ddts - preds) ** 2)

def accuracy(params, deals, ddts):
  predict_round = jnp.round(batched_predict(params, deals))
  return jnp.count_nonzero(jnp.isclose(predict_round, ddts))/jnp.size(ddts)

"""
@jit
def update(params, deals, ddts):
  grads = grad(loss)(params, deals, ddts)
  updates, opt_state = optimizer.update(grads, opt_state)
  return optax.apply_updates(params, updates)
"""
print(accuracy(params, test_deals, test_ddts))

0.0119688


In [ ]:
key = random.PRNGKey(0)
keys = random.split(key, config.EPOCHS)
print("START learning")

n_steps_per_epoch = math.ceil(config.TRAIN_SAMPLE_NUM / config.BATCH_SIZE)
for epoch in range(config.EPOCHS):
  start_time = time.time()
  train_ddts = random.permutation(keys[epoch], train_ddts)
  train_deals = random.permutation(keys[epoch], train_deals)
  train_loss_sum = 0
  for step in range(0, n_steps_per_epoch):
    batched_train_deals = train_deals[step * config.BATCH_SIZE: (step + 1) * config.BATCH_SIZE]
    batched_train_ddts = train_ddts[step * config.BATCH_SIZE: (step + 1) * config.BATCH_SIZE]
    train_loss = loss(params, batched_train_deals, batched_train_ddts)
    train_loss_sum += train_loss
    grads = grad(loss)(params, batched_train_deals, batched_train_ddts)
    updates, opt_state = optimizer.update(grads, opt_state)
    params =  optax.apply_updates(params, updates)
    metrics = {"train/train_loss": train_loss,
               "train/epoch": (step + 1 + n_steps_per_epoch * epoch) / n_steps_per_epoch}
    if step + 1 < n_steps_per_epoch:
      wandb.log(metrics)
  epoch_time = time.time() - start_time
  test_loss = loss(params, test_deals, test_ddts)
  test_accuracy = accuracy(params, test_deals, test_ddts)
  test_metrics = {"test/test_loss": test_loss,
                  "test/test_accuracy": test_accuracy}
  wandb.log({**metrics, **test_metrics})
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set loss {}".format(train_loss_sum / (n_steps_per_epoch)))
  print("Test set loss {}".format(test_loss))
  print("Test set accuracy {}".format(accuracy(params, test_deals, test_ddts)))

wandb.finish()
for i in range(10):
  print("ddt:")
  print(test_ddts[i])
  print("predict:")
  print(jnp.round(predict(params, test_deals[i])))

START learning
